In [1]:
import pandas as pd
from sqlalchemy import create_engine
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import pymysql 
pymysql.install_as_MySQLdb()

In [2]:
#read CSV files into pandas
csv_atbats = "data/atbats.csv"
csv_games = "data/games.csv"
csv_pitches = "data/pitches.csv"
csv_playernames = "data/player_names.csv"

In [3]:
atbats_data_df = pd.read_csv(csv_atbats)
atbats_data_df.head()

,ab_id,batter_id,event,g_id,inning,o,p_score,p_throws,pitcher_id,stand,top
0,2015000001,572761,Groundout,201500001,1,1,0,L,452657,L,True
1,2015000002,518792,Double,201500001,1,1,0,L,452657,L,True
2,2015000003,407812,Single,201500001,1,1,0,L,452657,R,True
3,2015000004,425509,Strikeout,201500001,1,2,0,L,452657,R,True
4,2015000005,571431,Strikeout,201500001,1,3,0,L,452657,L,True


In [4]:
games_data_df = pd.read_csv(csv_games)
games_data_df.head()

,g_id,attendance,away_final_score,away_team,date,elapsed_time,home_final_score,home_team,start_time,umpire_1B,umpire_2B,umpire_3B,umpire_HP,venue_name,weather,wind,delay
0,201500001,"35,055",3,sln,2015-04-05,3:04,0,chn,7:17 PM,Mark Wegner,Marty Foster,Mike Muchlinski,Mike Winters,Wrigley Field,"44 degrees, clear","7 mph, In from CF",NaN
1,201500002,"45,909",1,ana,2015-04-06,2:33,4,sea,1:12 PM,Ron Kulpa,Brian Knight,Vic Carapazza,Larry Vanover,Safeco Field,"54 degrees, cloudy","1 mph, Varies",NaN
2,201500003,"36,969",2,atl,2015-04-06,2:36,1,mia,4:22 PM,Laz Diaz,Chris Guccione,Cory Blaser,Jeff Nelson,Marlins Park,"80 degrees, partly cloudy","16 mph, In from CF",:16
3,201500004,"31,042",6,bal,2015-04-06,3:01,2,tba,3:12 PM,Ed Hickox,Paul Nauert,Mike Estabrook,Dana DeMuth,Tropicana Field,"72 degrees, dome","0 mph, None",NaN
4,201500005,"45,549",8,bos,2015-04-06,3:01,0,phi,3:08 PM,Phil Cuzzi,Tony Randazzo,Will Little,Gerry Davis,Citizens Bank Park,"71 degrees, partly cloudy","11 mph, Out to RF",NaN


In [5]:
pitches_data_df = pd.read_csv(csv_pitches)
pitches_data_df.head()

,ab_id,ax,ay,az,b_count,b_score,break_angle,break_length,break_y,code,...,type_confidence,vx0,vy0,vz0,x,x0,y,y0,z0,zone
0,2.015000e+09,7.665,34.685,-11.960,0,0,-25.0,3.2,23.7,C,...,2.0,-6.409,-136.065,-3.995,101.140000,2.280,158.780000,50.0,5.302,3.0
1,2.015000e+09,12.043,34.225,-10.085,0,0,-40.7,3.4,23.7,S,...,2.0,-8.411,-135.690,-5.980,124.280000,2.119,175.410000,50.0,5.307,5.0
2,2.015000e+09,14.368,35.276,-11.560,0,0,-43.7,3.7,23.7,F,...,2.0,-9.802,-137.668,-3.337,136.740000,2.127,150.110000,50.0,5.313,1.0
3,2.015000e+09,2.104,28.354,-20.540,0,0,-1.3,5.0,23.8,B,...,2.0,-8.071,-133.005,-6.567,109.685636,2.279,187.463482,50.0,5.210,13.0
4,2.015000e+09,-10.280,21.774,-34.111,1,0,18.4,12.0,23.8,B,...,2.0,-6.309,-110.409,0.325,146.527525,2.179,177.242829,50.0,5.557,13.0


In [6]:
playernames_data_df = pd.read_csv(csv_playernames)
playernames_data_df.head()

,id,first_name,last_name
0,452657,Jon,Lester
1,425794,Adam,Wainwright
2,457435,Phil,Coke
3,435400,Jason,Motte
4,519166,Neil,Ramirez


In [7]:
#create the url's for each player
playernames_data_df['name_url'] = playernames_data_df['last_name'].str[0:5].str.lower() + playernames_data_df['first_name'].str[0:2].str.lower()
playernames_data_df['name_url_edit'] = playernames_data_df['name_url'].str.replace("'","")
playernames_data_df['name_url_edit'] = playernames_data_df['name_url'].str.replace(" ","")
playernames_data_df['search_url'] =  "https://www.baseball-reference.com/players/" +playernames_data_df['name_url_edit'].str[0:1]+"/"+ playernames_data_df['name_url_edit'] + "01.shtml"



In [8]:
playernames_data_df.head()

,id,first_name,last_name,name_url,name_url_edit,search_url
0,452657,Jon,Lester,lestejo,lestejo,https://www.baseball-reference.com/players/l/l...
1,425794,Adam,Wainwright,wainwad,wainwad,https://www.baseball-reference.com/players/w/w...
2,457435,Phil,Coke,cokeph,cokeph,https://www.baseball-reference.com/players/c/c...
3,435400,Jason,Motte,motteja,motteja,https://www.baseball-reference.com/players/m/m...
4,519166,Neil,Ramirez,ramirne,ramirne,https://www.baseball-reference.com/players/r/r...


In [9]:
#See which url's connect successfully
connection = []
for index, row in playernames_data_df.iterrows(): 
    request = requests.get(playernames_data_df['search_url'][index])
    if request.status_code == 200:
        connection.append(1)
    else:
        connection.append(0)

In [10]:
#append connection column to existing dataframe
playernames_data_df['connection'] = connection

In [12]:
#remove 404 connection errors
#playernames_success_df = playernames_data_df.loc[(playernames_data_df['connection'] == 1),:]
playernames_success_df = playernames_data_df

In [13]:
#reindex
playernames_success_df = playernames_success_df.reset_index(drop=True)

In [14]:
    weight = []
    for index,row in playernames_success_df.iterrows(): 
        html = row['search_url']
        response = requests.get(html)
        soup = bs(response.text, 'html.parser')
        try:
            weight.append(soup.find('span', itemprop='weight').text)
        except(AttributeError):
            weight.append("NA")

In [15]:
#append weight data to existing dataframe
playernames_success_df['weight'] = weight

In [ ]:
#remove NA's
#playernames_success_df = playernames_success_df.loc[(playernames_success_df['weight'] != "NA"),:]


In [16]:
#reindex
playernames_success_df = playernames_success_df.reset_index(drop=True)

In [17]:
#drop the connection column
playernames_success_df = playernames_success_df.drop(['connection'],axis=1)

In [18]:
playernames_success_df.head()

,id,first_name,last_name,name_url,name_url_edit,search_url,weight
0,452657,Jon,Lester,lestejo,lestejo,https://www.baseball-reference.com/players/l/l...,240lb
1,425794,Adam,Wainwright,wainwad,wainwad,https://www.baseball-reference.com/players/w/w...,235lb
2,457435,Phil,Coke,cokeph,cokeph,https://www.baseball-reference.com/players/c/c...,210lb
3,435400,Jason,Motte,motteja,motteja,https://www.baseball-reference.com/players/m/m...,205lb
4,519166,Neil,Ramirez,ramirne,ramirne,https://www.baseball-reference.com/players/r/r...,215lb


In [19]:
#drop index from all dataframes
atbats_data_df.set_index(['ab_id'],inplace=True)
atbats_data_df.head()

,batter_id,event,g_id,inning,o,p_score,p_throws,pitcher_id,stand,top
ab_id,,,,,,,,,,
2015000001,572761,Groundout,201500001,1,1,0,L,452657,L,True
2015000002,518792,Double,201500001,1,1,0,L,452657,L,True
2015000003,407812,Single,201500001,1,1,0,L,452657,R,True
2015000004,425509,Strikeout,201500001,1,2,0,L,452657,R,True
2015000005,571431,Strikeout,201500001,1,3,0,L,452657,L,True


In [20]:
pitches_data_df.set_index(['ab_id'],inplace=True)
pitches_data_df.head()


,ax,ay,az,b_count,b_score,break_angle,break_length,break_y,code,end_speed,...,type_confidence,vx0,vy0,vz0,x,x0,y,y0,z0,zone
ab_id,,,,,,,,,,,,,,,,,,,,,
2.015000e+09,7.665,34.685,-11.960,0,0,-25.0,3.2,23.7,C,84.1,...,2.0,-6.409,-136.065,-3.995,101.140000,2.280,158.780000,50.0,5.302,3.0
2.015000e+09,12.043,34.225,-10.085,0,0,-40.7,3.4,23.7,S,84.1,...,2.0,-8.411,-135.690,-5.980,124.280000,2.119,175.410000,50.0,5.307,5.0
2.015000e+09,14.368,35.276,-11.560,0,0,-43.7,3.7,23.7,F,85.2,...,2.0,-9.802,-137.668,-3.337,136.740000,2.127,150.110000,50.0,5.313,1.0
2.015000e+09,2.104,28.354,-20.540,0,0,-1.3,5.0,23.8,B,84.0,...,2.0,-8.071,-133.005,-6.567,109.685636,2.279,187.463482,50.0,5.210,13.0
2.015000e+09,-10.280,21.774,-34.111,1,0,18.4,12.0,23.8,B,69.6,...,2.0,-6.309,-110.409,0.325,146.527525,2.179,177.242829,50.0,5.557,13.0


In [21]:
games_data_df.set_index(['g_id'],inplace=True)
games_data_df.head()

,attendance,away_final_score,away_team,date,elapsed_time,home_final_score,home_team,start_time,umpire_1B,umpire_2B,umpire_3B,umpire_HP,venue_name,weather,wind,delay
g_id,,,,,,,,,,,,,,,,
201500001,"35,055",3,sln,2015-04-05,3:04,0,chn,7:17 PM,Mark Wegner,Marty Foster,Mike Muchlinski,Mike Winters,Wrigley Field,"44 degrees, clear","7 mph, In from CF",NaN
201500002,"45,909",1,ana,2015-04-06,2:33,4,sea,1:12 PM,Ron Kulpa,Brian Knight,Vic Carapazza,Larry Vanover,Safeco Field,"54 degrees, cloudy","1 mph, Varies",NaN
201500003,"36,969",2,atl,2015-04-06,2:36,1,mia,4:22 PM,Laz Diaz,Chris Guccione,Cory Blaser,Jeff Nelson,Marlins Park,"80 degrees, partly cloudy","16 mph, In from CF",:16
201500004,"31,042",6,bal,2015-04-06,3:01,2,tba,3:12 PM,Ed Hickox,Paul Nauert,Mike Estabrook,Dana DeMuth,Tropicana Field,"72 degrees, dome","0 mph, None",NaN
201500005,"45,549",8,bos,2015-04-06,3:01,0,phi,3:08 PM,Phil Cuzzi,Tony Randazzo,Will Little,Gerry Davis,Citizens Bank Park,"71 degrees, partly cloudy","11 mph, Out to RF",NaN


In [22]:
playernames_success_df.set_index(['id'],inplace=True)
playernames_success_df.head()

,first_name,last_name,name_url,name_url_edit,search_url,weight
id,,,,,,
452657,Jon,Lester,lestejo,lestejo,https://www.baseball-reference.com/players/l/l...,240lb
425794,Adam,Wainwright,wainwad,wainwad,https://www.baseball-reference.com/players/w/w...,235lb
457435,Phil,Coke,cokeph,cokeph,https://www.baseball-reference.com/players/c/c...,210lb
435400,Jason,Motte,motteja,motteja,https://www.baseball-reference.com/players/m/m...,205lb
519166,Neil,Ramirez,ramirne,ramirne,https://www.baseball-reference.com/players/r/r...,215lb


In [23]:
#connect to local database
rds_connection_string = "root:Banking5%@127.0.0.1/baseball_db"
engine = create_engine(f'mysql://{rds_connection_string}')

In [24]:
#check for tables
engine.table_names()

['atbats', 'games', 'pitches', 'player_names']

In [25]:
print(pd.io.sql.get_schema(pitches_data_df.reset_index(), 'pitches'))

CREATE TABLE "pitches" (
"ab_id" REAL,
  "ax" REAL,
  "ay" REAL,
  "az" REAL,
  "b_count" INTEGER,
  "b_score" INTEGER,
  "break_angle" REAL,
  "break_length" REAL,
  "break_y" REAL,
  "code" TEXT,
  "end_speed" REAL,
  "nasty" REAL,
  "on_1b" INTEGER,
  "on_2b" INTEGER,
  "on_3b" INTEGER,
  "outs" INTEGER,
  "pfx_x" REAL,
  "pfx_z" REAL,
  "pitch_num" INTEGER,
  "pitch_type" TEXT,
  "px" REAL,
  "pz" REAL,
  "s_count" INTEGER,
  "spin_dir" REAL,
  "spin_rate" REAL,
  "start_speed" REAL,
  "sz_bot" REAL,
  "sz_top" REAL,
  "type" TEXT,
  "type_confidence" REAL,
  "vx0" REAL,
  "vy0" REAL,
  "vz0" REAL,
  "x" REAL,
  "x0" REAL,
  "y" REAL,
  "y0" REAL,
  "z0" REAL,
  "zone" REAL
)


In [ ]:
print(pd.io.sql.get_schema(atbats_data_df.reset_index(), 'atbats'))

In [ ]:
print(pd.io.sql.get_schema(games_data_df.reset_index(), 'games'))

In [ ]:
print(pd.io.sql.get_schema(playernames_success_df.reset_index(), 'player_names'))

In [26]:
atbats_data_df.to_sql(name='atbats', con=engine, if_exists='replace', index=True)

In [27]:
playernames_success_df.to_sql(name='player_names', con=engine, if_exists='replace', index=True)


In [28]:
games_data_df.to_sql(name='games', con=engine, if_exists='replace', index=True)

In [30]:
pitches_data_df.to_sql(name='pitches', con=engine, if_exists='replace', index=True)

In [29]:
pd.read_sql_query('select * from games', con=engine).head()

,g_id,attendance,away_final_score,away_team,date,elapsed_time,home_final_score,home_team,start_time,umpire_1B,umpire_2B,umpire_3B,umpire_HP,venue_name,weather,wind,delay
0,201500001,"35,055",3,sln,2015-04-05,3:04,0,chn,7:17 PM,Mark Wegner,Marty Foster,Mike Muchlinski,Mike Winters,Wrigley Field,"44 degrees, clear","7 mph, In from CF",None
1,201500002,"45,909",1,ana,2015-04-06,2:33,4,sea,1:12 PM,Ron Kulpa,Brian Knight,Vic Carapazza,Larry Vanover,Safeco Field,"54 degrees, cloudy","1 mph, Varies",None
2,201500003,"36,969",2,atl,2015-04-06,2:36,1,mia,4:22 PM,Laz Diaz,Chris Guccione,Cory Blaser,Jeff Nelson,Marlins Park,"80 degrees, partly cloudy","16 mph, In from CF",:16
3,201500004,"31,042",6,bal,2015-04-06,3:01,2,tba,3:12 PM,Ed Hickox,Paul Nauert,Mike Estabrook,Dana DeMuth,Tropicana Field,"72 degrees, dome","0 mph, None",None
4,201500005,"45,549",8,bos,2015-04-06,3:01,0,phi,3:08 PM,Phil Cuzzi,Tony Randazzo,Will Little,Gerry Davis,Citizens Bank Park,"71 degrees, partly cloudy","11 mph, Out to RF",None
